In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
from bs4 import BeautifulSoup 
import requests

from RGGI_plant_analysis import RGGI_capacity
from Analyse_PJM_capacity import run_historical_RGGI_share,run_future_RGGI_share

In [9]:
import sys
from statistics import NormalDist
import subprocess
import os 

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

from enviro_vars import set_environmental_variables
try:       
    set_environmental_variables()
except Exception as e:
    print(e)
    print('in online environment - no enviro_vars module')

############# ADD GIT TOKEN BELOW ############
git_token = os.environ['gittoken']
AUTHORITY = os.environ['azAuth']
CLIENT_ID = os.environ['azClientid']
SECRET = os.environ['AzAppSecret']
location =  {"authority": AUTHORITY,"client_id": CLIENT_ID,'secret':SECRET,"scope": [ "https://graph.microsoft.com/.default" ]}

##
github_repository = f"git+https://{git_token}@github.com/research-carbon-cap/SharePointv2.git@Azure_functions"

install(github_repository)


from SharePointv2.Sharepoint_API import GRAPH_API

Environment variables set!


In [10]:
def run_full_relative_capacity_PJM(read_latest):
    historical_rggi_share_pjm,test_df = run_historical_RGGI_share(read_latest=read_latest)
    historical_rggi_share_pjm['RGGI_share'] = historical_rggi_share_pjm[1]/historical_rggi_share_pjm[[1,0]].sum(axis=1)
    rggi_pjm_rggi_plus_retirements_plus_additions_timeseries,pjm_plus_retirements_plus_additions_timeseries,PJM_plants =  run_future_RGGI_share()
    rggi_share_pjm = pjm_plus_retirements_plus_additions_timeseries.iloc[1:].stack().to_frame('PJM').join(rggi_pjm_rggi_plus_retirements_plus_additions_timeseries.iloc[1:].stack().to_frame('RGGI_PJM'))
    rggi_share_pjm = rggi_share_pjm.reset_index()

    rggi_share_pjm['RGGI_share'] = rggi_share_pjm['RGGI_PJM']/rggi_share_pjm['PJM']
    rggi_share_pjm['report_month'] = rggi_share_pjm.level_0.dt.month
    rggi_share_pjm['report_year'] = rggi_share_pjm.level_0.dt.year
    rggi_share_pjm['level_0'] = rggi_share_pjm['level_0']+pd.offsets.MonthEnd(0)
    rggi_share_pjm = rggi_share_pjm.rename(columns={'level_1':'PJM_tech',
                                                    'level_0':'Date'})
    historical_and_forecast = pd.concat([historical_rggi_share_pjm,rggi_share_pjm[['Date','PJM_tech','RGGI_share','report_month','report_year']]],axis=0,join='outer')#.to_csv('RGGI_share_PJM.csv')

    return historical_and_forecast

In [11]:
historical_rggi_share_pjm,test_df = run_historical_RGGI_share(location,read_latest=True)

In [43]:
historical_rggi_share_pjm['RGGI_share_PJM'] = historical_rggi_share_pjm[1]/historical_rggi_share_pjm[[0,1]].sum(axis=1)

In [45]:
historical_rggi_share_pjm

RGGI_eligible,report_year,report_month,PJM_tech,0,1,Date,RGGI_share_PJM
0,2018,12,Coal,40965.7,2191.3,2018-12-31,0.050775
1,2018,12,Gas,64823.8,17414.0,2018-12-31,0.211752
2,2018,12,Hydro,2476.3,530.8,2018-12-31,0.176516
3,2018,12,Multiple Fuels,27902.1,10523.0,2018-12-31,0.273857
4,2018,12,Nuclear,27979.1,3119.4,2018-12-31,0.100307
...,...,...,...,...,...,...,...
776,2024,10,Other,509.2,242.8,2024-10-31,0.322872
777,2024,10,Other Renewables,1073.9,NaN,2024-10-31,NaN
778,2024,10,Solar,12934.6,259.1,2024-10-31,0.019638
779,2024,10,Storage,4244.0,453.0,2024-10-31,0.096445


In [47]:
compare = historical_rggi_share_pjm.groupby(['Date','PJM_tech'])['RGGI_share_PJM'].mean().unstack('PJM_tech')

In [52]:
fig= px.line(compare,labels={'value':'RGGI-PJM Share of PJM Capacity (%)'})
fig.update_layout(yaxis_tickformat='.0%')

In [40]:
(compare[1]/compare['Total PJM']).mean()

0.14928162496635305

In [28]:
px.line(compare.rename(columns={1:'RGGI-PJM'}).iloc[:,-2:],labels={'value':'Capacity (MW)'})

In [54]:
test_df['date'] = pd.to_datetime('01/'+test_df['report_month'].astype(str)+'/'+test_df['report_year'].astype(str))

In [55]:
historical_rggi_share_pjm['RGGI_share'] = historical_rggi_share_pjm[1]/historical_rggi_share_pjm[[1,0]].sum(axis=1)

In [60]:
rggi_pjm_rggi_plus_retirements_plus_additions_timeseries,pjm_plus_retirements_plus_additions_timeseries,PJM_plants =  run_future_RGGI_share(location)

saving down data for 2024-09-01
Operating
Canceled or Postponed
Operating_PR
Planned
Planned_PR
Retired
Retired_PR


c:\GitHub\RGGI_capacity_data\Analyse_PJM_capacity.py:94: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\GitHub\RGGI_capacity_data\Analyse_PJM_capacity.py:95: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\GitHub\RGGI_capacity_data\PJM_retirements.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [61]:
rggi_share_pjm = pjm_plus_retirements_plus_additions_timeseries.iloc[1:].stack().to_frame('PJM').join(rggi_pjm_rggi_plus_retirements_plus_additions_timeseries.iloc[1:].stack().to_frame('RGGI_PJM'))
rggi_share_pjm = rggi_share_pjm.reset_index()


In [62]:
rggi_share_pjm['RGGI_share'] = rggi_share_pjm['RGGI_PJM']/rggi_share_pjm['PJM']

In [63]:
rggi_share_pjm['report_month'] = rggi_share_pjm.level_0.dt.month
rggi_share_pjm['report_year'] = rggi_share_pjm.level_0.dt.year

In [64]:
rggi_share_pjm['level_0'] = rggi_share_pjm['level_0']+pd.offsets.MonthEnd(0)

In [65]:
rggi_share_pjm = rggi_share_pjm.rename(columns={'level_1':'PJM_tech',
                                                'level_0':'Date'})

In [66]:
historical_and_forecast = pd.concat([historical_rggi_share_pjm,rggi_share_pjm[['Date','PJM_tech','RGGI_share','report_month','report_year']]],axis=0,join='outer')#.to_csv('RGGI_share_PJM.csv')

In [67]:
historical_and_forecast.groupby(['report_year','report_month','Date','PJM_tech'])['RGGI_share'].mean().to_frame().fillna(0).to_csv('RGGI_share_PJM.csv')

In [68]:
historical_and_forecast.groupby(['report_year','report_month','Date','PJM_tech'])['RGGI_share'].mean().to_frame().fillna(0).loc[2024,9]#.query('report_year==2024 and report_month==9')

RGGI_share
Date       PJM_tech                    
2024-09-30 Coal                0.044130
           Gas                 0.173462
           Hydro               0.176246
           Multiple Fuels      0.227322
           Nuclear             0.104123
           Oil                 0.306837
           Other               0.322872
           Other Renewables    0.000000
           Solar               0.020405
           Storage             0.096445
           Wind                0.016583

In [69]:
rggi_share_pjm.query('report_month==11 and report_year==2024')

,Date,PJM_tech,PJM,RGGI_PJM,RGGI_share,report_month,report_year
22,2024-11-30,Coal,31049.5,1370.2,0.044130,11,2024
23,2024-11-30,Gas,94952.1,16470.6,0.173462,11,2024
24,2024-11-30,Hydro,3011.7,530.8,0.176246,11,2024
25,2024-11-30,Multiple Fuels,25764.8,5856.9,0.227322,11,2024
26,2024-11-30,Nuclear,30167.2,3141.1,0.104123,11,2024
27,2024-11-30,Oil,5082.5,1559.5,0.306837,11,2024
28,2024-11-30,Other,752.0,242.8,0.322872,11,2024
29,2024-11-30,Other Renewables,1076.7,0.0,0.000000,11,2024
30,2024-11-30,Solar,14345.2,259.1,0.018062,11,2024
31,2024-11-30,Storage,4697.0,453.0,0.096445,11,2024


In [70]:
historical_and_forecast#.query('report_year==2023 and report_month==3')

,report_year,report_month,PJM_tech,0,1,Date,RGGI_share_PJM,RGGI_share
0,2018,12,Coal,40965.7,2191.3,2018-12-31,0.050775,0.050775
1,2018,12,Gas,64823.8,17414.0,2018-12-31,0.211752,0.211752
2,2018,12,Hydro,2476.3,530.8,2018-12-31,0.176516,0.176516
3,2018,12,Multiple Fuels,27902.1,10523.0,2018-12-31,0.273857,0.273857
4,2018,12,Nuclear,27979.1,3119.4,2018-12-31,0.100307,0.100307
...,...,...,...,...,...,...,...,...
831,2030,12,Other,NaN,NaN,2030-12-31,NaN,0.322872
832,2030,12,Other Renewables,NaN,NaN,2030-12-31,NaN,0.000000
833,2030,12,Solar,NaN,NaN,2030-12-31,NaN,0.039280
834,2030,12,Storage,NaN,NaN,2030-12-31,NaN,0.087082


In [72]:
fig = px.line(historical_and_forecast,x='Date',y='RGGI_share',color='PJM_tech')
fig.update_layout(yaxis_tickformat='.0%')